In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from google.colab import drive
from keras.utils import to_categorical
from sklearn.metrics import f1_score, mean_squared_error
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from collections import Counter

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_titanic = pd.read_csv('/content/drive/MyDrive/train.csv')
df_boston = pd.read_csv('/content/drive/MyDrive/BostonHousing.csv')

In [ ]:
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df_titanic_factorized = pd.DataFrame(df_titanic)
df_titanic = df_titanic_factorized
for column in df_titanic_factorized.select_dtypes(include=['object']).columns:
    df_titanic_factorized[column], unique = pd.factorize(df_titanic_factorized[column])

In [5]:
df_titanic_factorized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    int64  
 4   Sex          891 non-null    int64  
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    int64  
 9   Fare         891 non-null    float64
 10  Cabin        891 non-null    int64  
 11  Embarked     891 non-null    int64  
dtypes: float64(2), int64(10)
memory usage: 83.7 KB


# Titanic

In [ ]:
X = df_titanic_factorized.iloc[:, [2, 4, 5, 6, 7, 9, 11]]
y = df_titanic_factorized.iloc[:, 1]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state=3, stratify=y)

In [ ]:
for k in range(1, 100):
    for i in range(1, 30):
        model = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=6, max_features=6, max_depth=5, random_state=5)
        model.fit(X_train, y_train)

        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)

        f1_train = f1_score(y_pred_train, y_train)
        f1_test = f1_score(y_pred_test, y_test)
        if f1_test > 0.78 and f1_train - f1_test > 0:
            print(f'{k} {i}: {f1_test}, {f1_train - f1_test}')

1 7: 0.7887323943661971, 0.006139400505597692
1 8: 0.7887323943661971, 0.006139400505597692
1 9: 0.7887323943661971, 0.006139400505597692
1 10: 0.7887323943661971, 0.006139400505597692
1 11: 0.7887323943661971, 0.006139400505597692
1 12: 0.7887323943661971, 0.006139400505597692
1 13: 0.7887323943661971, 0.006139400505597692
1 14: 0.7887323943661971, 0.006139400505597692
1 15: 0.7887323943661971, 0.006139400505597692
1 16: 0.7887323943661971, 0.006139400505597692
1 17: 0.7887323943661971, 0.006139400505597692
1 18: 0.7887323943661971, 0.006139400505597692
1 19: 0.7887323943661971, 0.006139400505597692
1 20: 0.7887323943661971, 0.006139400505597692
1 21: 0.7887323943661971, 0.006139400505597692
1 22: 0.7887323943661971, 0.006139400505597692
1 23: 0.7887323943661971, 0.006139400505597692
1 24: 0.7887323943661971, 0.006139400505597692
1 25: 0.7887323943661971, 0.006139400505597692
1 26: 0.7887323943661971, 0.006139400505597692
1 27: 0.7887323943661971, 0.006139400505597692
1 28: 0.78873239

In [ ]:
f1_train = 0
f1_test = 0

for i in range(10,30):
    for k in range(100,200):
                model = RandomForestClassifier(max_depth=i, n_estimators=k, random_state=5) #best: max_depth = 10, n_estimators = 197
                model.fit(X_train, y_train)

                y_pred_train = model.predict(X_train)
                y_pred_test = model.predict(X_test)

                new_f1_train = f1_score(y_pred_train, y_train)
                new_f1_test = f1_score(y_pred_test, y_test)

                if new_f1_test > f1_test:
                    print(f'{i}, {k}: {new_f1_train}, {new_f1_test}')
                    f1_train = new_f1_train
                    f1_test = new_f1_test

10, 100: 0.9239766081871345, 0.8201438848920863
10, 101: 0.9239766081871345, 0.8260869565217391
10, 122: 0.9278752436647173, 0.8345323741007195
10, 177: 0.923679060665362, 0.8405797101449275
10, 196: 0.92578125, 0.8467153284671532
10, 197: 0.923679060665362, 0.8529411764705882


KeyboardInterrupt: 

# Boston

In [ ]:
X = df_boston.drop('medv', axis=1)
y = df_boston['medv']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state=3)

In [ ]:
mse_train = 100
mse_test = 100

for i in range(1,20):
    for k in range(1,20):
        for l in range(1,20):
            model = DecisionTreeRegressor(min_samples_leaf=i, max_features=k, max_depth=l, random_state=5)
            model.fit(X_train, y_train)

            y_pred_train = model.predict(X_train)
            y_pred_test = model.predict(X_test)

            new_mse_train = mean_squared_error(y_pred_train, y_train)
            new_mse_test = mean_squared_error(y_pred_test, y_test)

            if new_mse_test < mse_test:
                print(f'{i}, {k}, {l}: {new_mse_train}, {new_mse_test}')
                mse_train = new_mse_train
                mse_test = new_mse_test

1, 1, 1: 74.58372145567922, 75.22351942233728
1, 1, 3: 58.3516708875364, 60.68037518111523
1, 1, 4: 36.33845570997264, 28.128932758269674
1, 1, 12: 0.9022716878830739, 26.281656815503975
1, 2, 10: 1.366816330740217, 16.77180317210217
1, 3, 7: 4.347777409726267, 14.127521069546077
1, 3, 8: 4.529527573394808, 10.417512003556784
1, 6, 7: 3.4400425061453563, 10.334337002861965
1, 9, 5: 7.208348044925767, 10.073198887047923
1, 13, 5: 6.879787791470963, 9.808548465232468
2, 11, 9: 3.549059796769151, 9.736516135073986
3, 10, 8: 6.037270471807451, 8.815639543118921
5, 9, 8: 6.574273723996964, 8.762433013318434
7, 9, 8: 11.485698991327705, 8.277200293209185


In [ ]:
mse_train = 100
mse_test = 100

for i in range(1,20):
    for k in range(15,50):
            model = RandomForestRegressor(max_depth=i, n_estimators=k, random_state=5) # best: max_depth=9, n_estimators=30
            model.fit(X_train, y_train)

            y_pred_train = model.predict(X_train)
            y_pred_test = model.predict(X_test)

            new_mse_train = mean_squared_error(y_pred_train, y_train)
            new_mse_test = mean_squared_error(y_pred_test, y_test)

            if new_mse_test < mse_test:
                print(f'{i}, {k}, {l}: {new_mse_train}, {new_mse_test}')
                mse_train = new_mse_train
                mse_test = new_mse_test

1, 29, 12: 1.8993040596270807, 7.700075388310268
2, 29, 12: 2.0520329743692556, 7.282895374738282
3, 29, 12: 1.6929958703415504, 7.749224011133106
4, 29, 12: 2.5695807370009383, 9.084475229485829
5, 29, 12: 2.568319733886851, 7.038050911388788
6, 29, 12: 2.0070623084117116, 7.42427347210836
7, 29, 12: 2.0357980308261894, 7.801569171802009
8, 29, 12: 2.145980577222285, 7.969454321676185
9, 29, 12: 2.299946575290105, 8.257121242930038
10, 29, 12: 2.0947510720268006, 7.685303113242383
11, 29, 12: 2.2407369358549243, 7.56851134977313
12, 29, 12: 1.9537112196773339, 7.467636164312275
13, 29, 12: 2.262554097819539, 7.393735805858085
14, 29, 12: 1.8247093560548433, 7.467475382992663
15, 29, 12: 1.9396781065115076, 7.732268239141473
16, 29, 12: 1.9703924002203559, 7.559115559302268
17, 29, 12: 2.054780306906274, 8.857397427111351
18, 29, 12: 2.169078527855856, 8.576409393466376
19, 29, 12: 2.06798892889908, 7.185042250770871
20, 29, 12: 2.269334988868654, 7.284423853566605
21, 29, 12: 2.012077

In [254]:
class MyDecisionTree:

    def __init__(self, X: list, y: list):
        self.X = X
        self.y = y

    def counter(self) -> list:
        counter = []
        for i in range(len(X)):
            counter.append([X[i], y[i]])
        return counter

    def decision(self) -> None:
        counter = self.counter()
        counter_sum_value = 100
        for i in range(counter[0][0]+1, counter[-1][0]+2):
            counter_1 = []
            counter_2 = []
            counter_sum = []
            for k in range(len(counter)):
                if counter[k][0] < i:
                    counter_1.append(counter[k])
                    counter_1_y = []
                    counter_1_y_fin = []
                    for l in range(len(counter_1)):
                        counter_1_y.append(counter_1[l][1])
                        counter_1_y_fin.append(np.mean(counter_1_y)%1)
                    counter_sum.append(np.sum(counter_1_y_fin))
                else:
                    counter_2.append(counter[k])
                    counter_2_y = []
                    counter_2_y_fin = []
                    for l in range(len(counter_2)):
                        counter_2_y.append(counter_2[l][1])
                        counter_2_y_fin.append(np.mean(counter_2_y)%1)
                    counter_sum.append(np.sum(counter_2_y_fin))
            new_counter_sum_value = np.sum(counter_sum)
            if new_counter_sum_value < counter_sum_value:
                best = i
                counter_sum_value = new_counter_sum_value

        print(f'Значение Х для лучшего разбиения: {best}')

In [259]:
X = [1, 3, 5, 7]
y = [0, 1, 1, 1]

lst = MyDecisionTree(X, y)

lst.decision()

Значение Х для лучшего разбиения: 2


In [260]:
X = [3, 5, 6, 7]
y = [0, 0, 0, 1]

lst = MyDecisionTree(X, y)

lst.decision()

Значение Х для лучшего разбиения: 7
